## ROS2 Migration from ROS1

### package.xml


In [ ]:
<!-- <package format="1"> -->
<package format="3">
<!-- ROS1 支持所有的版本1，2，3。而ROS2只支持2或者更高的版本 -->


<!-- 移除 -->
<metapackages/>


<!-- message 的格式要变一下 -->
Message.msg ==> msg/Message.msg

<!--  <buildtool_depend>catkin</buildtool_depend> -->
  <buildtool_depend>ament_cmake</buildtool_depend>


<depend>rclcpp</depend>
<depend>std_msgs</depend>



<!-- 自定义包的时候-->
<buildtool_depend>rosidl_default_generators</buildtool_depend>
<exec_depend>rosidl_default_runtime</exec_depend>


<export>
  <build_type>ament_cmake</build_type>
</export>


### CMakeList.txt

In [ ]:
# cmake_minimum_required(VERSION 2.8.3)
cmake_minimum_required(VERSION 3.5)


# 加上
set(CMAKE_CXX_STANDARD 14)

# 自定义包的时候
find_package(rosidl_default_generators REQUIRED)
# 载入的包
find_package(message_package REQUIRED)

# 将 generate_messages 方程改成:
rosidl_generate_interfaces 代替 add_message_files 和 add_service_files 这两个函数


# 系统编译 Build System
# 编译系统是ament 而编译工具是 colcon。
# ament_cmake 提供CMake方程,使得CMakeList.txt 编写更加方便。


find_package(ament_cmake REQUIRED)
find_package(rclcpp REQUIRED)
# ...
find_package(std_msgs REQUIRED)

# system Dependencies
find_package(Boost REQUIRED COMPONENTS system filesystem thread)

# CATKIN_DEPENDS    export dependencies
ament_export_dependencies(...)

# INCLUDE_DIRS
ament_export_include_directories(...)

# LIBRARIES
ament_export_libraries(...)


# include_directories(${catkin_INCLUDE_DIRS})
include_directories(include ${Boost_INCLUDE_DIRS})


# install(TARGETS talker
#  RUNTIME DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION})
install(TARGETS talker
  DESTINATION lib/${PROJECT_NAME})


# target_link_libraries(talker ${catkin_LIBRARIES})
ament_target_dependencies(talker
  rclcpp
  std_msgs)


target_include_directories(target include ${Boost_INCLUDE_DIRS})

# At the bottom of the file:
ament_package()


### CODE 代码端的修改

In [ ]:
// 中间加上了/msg/
// CamelCase to underscore separation 从驼峰型变成了小写加下划线类型
// .h 变成了 .hpp
#include <my_interfaces/msg/my_message.hpp>
//#include "std_msgs/time.h"
#include "builtin_interfaces/msg/time.hpp"

//#include "ros/ros.h"
#include "rclcpp/rclcpp.hpp"


// 实现的时候,仍然是驼峰型
my_interfaces::msg::MyMessage;
my_interfaces::msg::MyMessage::SharedPtr;
my_interfaces::msg::MyMessage::ConstSharedPtr;


//ros::Time
rclcpp::Time;
// Time 里面的 nsec => nanosec

//ros::Rate
rclcpp::Rate;

// 尽量不要使用boost。因为很多已经被纳入c++标准库里面了，所以尽可能脱离对boost的依赖

//  ros::init(argc, argv, "talker");
//  ros::NodeHandle n;
rclcpp::init(argc, argv);
auto node = rclcpp::Node::make_shared("talker");


//  ros::Publisher chatter_pub = n.advertise<std_msgs::String>("chatter", 1000);
//  ros::Rate loop_rate(10);
auto chatter_pub = node->create_publisher<std_msgs::msg::String>("chatter",1000);
rclcpp::Rate loop_rate(10);

//  while (ros::ok())
  while (rclcpp::ok())


  //  std_msgs::String msg;
std_msgs::msg::String msg;
msg.data = ss.str();

  //    ROS_INFO("%s", msg.data.c_str());
    RCLCPP_INFO(node->get_logger(), "%s\n", msg.data.c_str());

//发布消息
chatter_pub->publish(msg);


// ros::spinOnce();
rclcpp::spin_some(node);

### Parameters
-   [YAML file from ROS1 to ROS2](https://docs.ros.org/en/foxy/How-To-Guides/Parameters-YAML-files-migration-guide.html)

-   ROS2 里面的参数都是根据不同的节点定义的。 所以YAML文件，会差很多！

### Launch File

-   [Launch file from ROS1 to ROS2](https://docs.ros.org/en/foxy/How-To-Guides/Launch-files-migration-guide.html)

### 自定义msg和srv
-   [自定义类型和使用](https://docs.ros.org/en/foxy/Tutorials/Custom-ROS2-Interfaces.html)

#### Pure Python packages 纯用python编写的话:
只引用setup.py

- package.xml


```
      <export>
        <build_type>ament_python</build_type>
      </export>
```

- 删除 CMakeList.txt
- 修改成 python3 格式

##  ros::Time ros::Duration ros::Timer ros::Rate
-   [ros时间总结](https://blog.csdn.net/u013834525/article/details/83863992)

- ros::Time and ros::Duration has the same definition
```
            int32 sec
            int32 nsec
```

1. Time 和 Duration 的使用
2. 延时与寻黄
3. 定时器 ros::Timer